# IMPORTS:

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist,squareform
from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from SampleLists.SampleFileFunctions import *

import os
from Utils import cacheOnDisk
from queue.qp import qp,fakeqp
from addloglevels import sethandlers

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
stop
done
done


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'30052018'

# PREPARE SEQUENCE MATRIX for distance correlation analysis:

In [6]:
#define parameters:

datasetName='PNP515_ss15000_rep2'
dataset_folder='%s/TCR_real_data/SubSampled15000data_rep2' %MyPath #change as necessary
minNshared=2
onlyProductive=True



In [ ]:
# check if the desired shared seq matrix exist and if not - generate it:

file1='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA_onlyProductive%s' %(dataset_folder,datasetName,minNshared,
                                                                               onlyProductive)
                                                                                 

if isfile(file1):
    print 'loading TCRdfRA...'
    TCRdfRA=pd.read_pickle(file1)
else:
    print 'generating TCRdfRA file...'
    print 'minNshared=%s' %minNshared
    print 'data set name = %s' %datasetName
    print 'only Productive = %s' %onlyProductive  
    minNsharedList=list(minNshared)
    gen_sharingMatrix_perDataset(datasetName,dataset_folder,minNsharedList,extractUniqueAA=True,
                                 getSharingStatistics=True,onlyProductive=onlyProductive)



In [ ]:
TCRdfRA.head()

## divide by 100 to get relative abundances and not percentage:

In [ ]:
TCRdfRA.sum().sum()

In [ ]:
TCRdfRA=TCRdfRA.div(100)

In [ ]:
TCRdfRA.sum().sum()

## normalize the TCRdf

In [ ]:
TCRdfRA.sum().sum()

In [ ]:
normTCRdfRA=TCRdfRA.div(TCRdfRA.sum(axis=1),axis=0)

In [ ]:
normTCRdfRA.sum(axis=1).head()

In [ ]:
file2='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA_onlyProductive%s_normed' %(dataset_folder,datasetName,minNshared,
                                                                               onlyProductive)
normTCRdfRA.to_pickle(file2)

## only more than 1 copy:

In [ ]:
TCRdfRA[TCRdfRA>0].min().min()

In [ ]:
TCRdfRA2copies=TCRdfRA.replace(TCRdfRA[TCRdfRA>0].min().min(),0)

In [ ]:
checkShared=TCRdfRA2copies.apply(lambda x: True if len([i for i in x if i>0])>1 else False)
shared=checkShared[checkShared]
sharedTCRdfRA2copies=TCRdfRA2copies[shared.index]
print sharedTCRdfRA2copies.shape
sharedTCRdfRA2copies.head()

In [ ]:
file3='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA_onlyProductive%s_2copiesshared' %(dataset_folder,datasetName,minNshared,
                                                                               onlyProductive)
sharedTCRdfRA2copies.to_pickle(file3)

In [ ]:
TCRdfRA2copiesshared_normed=sharedTCRdfRA2copies.div(sharedTCRdfRA2copies.sum(axis=1),axis=0)

In [ ]:
TCRdfRA2copiesshared_normed.sum(axis=1).head()

In [ ]:
file2='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA_onlyProductive%s_2copiesshared_normed' %(dataset_folder,datasetName,minNshared,
                                                                               onlyProductive)
TCRdfRA2copiesshared_normed.to_pickle(file2)

## no capping was done
the minimal freq possible is 0.000067 (1/150000). 

## remove outliers (std>5)

In [ ]:
# TrimmedTCRdf=filter_outliers(df=TCRdfRA,outlierSTD=5,columnList=None,trim=True)

In [ ]:
# TCRdfRA.sum().sum()

In [ ]:
# TrimmedTCRdf.sum().sum()

In [ ]:
# TrimmedTCRdf.sum().sum()

In [ ]:
#TrimmedTCRdf.head()

In [ ]:
# TrimmedTCRdf.index=TCRdfRA.index
# TrimmedTCRdf.head()

## save files

In [ ]:
# file2='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA_onlyProductive%s_divided_trimmed' %(dataset_folder,datasetName,minNshared,
#                                                                                onlyProductive)
# TrimmedTCRdf.to_pickle(file2)
                                                                                 

## optional: normalize df to 1:

In [ ]:
# normTrimmedTCRdf=TrimmedTCRdf.div(TrimmedTCRdf.sum(axis=1),axis=0)

In [ ]:
# TrimmedTCRdf.sum(axis=1).head()

In [ ]:
# normTrimmedTCRdf.sum(axis=1).head()

### save file again:

In [ ]:
# file3='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA_onlyProductive%s_divided_trimmed_normed' %(dataset_folder,datasetName,minNshared,
#                                                                                onlyProductive)
# normTrimmedTCRdf.to_pickle(file3)

## optional: cap at minValue*2
leave only sequences that appear at least in one sample in more than 1 copy:

In [ ]:
1.0/15000

In [ ]:
cappedTCRdfRA=TCRdfRA.mask(TCRdfRA<0.000067)
cappedTCRdfRA=cappedTCRdfRA.fillna(0.000067)
cappedTCRdfRA.head()

In [ ]:
checkShared=cappedTCRdfRA.apply(lambda x: True if len([i for i in x if i>0.000067])>1 else False)
shared=checkShared[checkShared]
sharedTCRdfRA=cappedTCRdfRA[shared.index]
print sharedTCRdfRA.shape
sharedTCRdfRA.head()

### trim the capped df:

In [ ]:
TrimmedSharedTCRdf=filter_outliers(df=sharedTCRdfRA,outlierSTD=5,columnList=None,trim=True)

In [ ]:
TrimmedSharedTCRdf.head()

In [ ]:
sharedTCRdfRA.sum().sum()

In [ ]:
TrimmedSharedTCRdf.sum().sum()

In [ ]:
file3='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA_onlyProductive%s_divided_trimmed_capped' %(dataset_folder,datasetName,minNshared,
                                                                               onlyProductive)
TrimmedSharedTCRdf.to_pickle(file3)

### normalize the trimmed capped TCRdf:

In [ ]:
normTrimmedSharedTCRdf=TrimmedSharedTCRdf.div(TrimmedSharedTCRdf.sum(axis=1),axis=0)

In [ ]:
TrimmedSharedTCRdf.sum(axis=1).head()

In [ ]:
normTrimmedSharedTCRdf.sum(axis=1).head()

In [ ]:
normTrimmedSharedTCRdf.head()

In [ ]:
file4='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA_onlyProductive%s_divided_trimmed_capped_normed' %(dataset_folder,datasetName,minNshared,
                                                                               onlyProductive)
normTrimmedSharedTCRdf.to_pickle(file4)

# prepare Mb matrix for distance correlation analysis:

## prepare MbDF table (mergedOnBD using noOutlierMean

In [ ]:
#load PNP515 sample list:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP515','rb') as fp:
    PNP515=pickle.load(fp)
    
#DEFINE PARAMETERS:
mbLevel='s'
mbDataFolder='AllSeqProjects'
SampleList=PNP515
SampleListName='PNP515'
filterGenotek=True
filterMinimalReads=9000000
filterlibPrepMethod=None
groupFunction='noOutlierMean'
nSTD=5
nMinSamples=3

the function genFilterMergeMbData was copied to TCR_microbiome_interactions_functions.py

In [ ]:
groupedFilteredMB=genFilterMergeMbData(mbLevel,mbDataFolder,SampleList,SampleListName,filterGenotek,filterMinimalReads,filterlibPrepMethod,
                        groupFunction,nSTD,nMinSamples)

In [ ]:
groupedFilteredMB.head()

## normalize MbDFold:

In [ ]:
groupedFilteredMB.sum().sum()

In [ ]:
groupedFilteredMB.head()

In [ ]:
normMbDF=groupedFilteredMB.div(groupedFilteredMB.sum(axis=1),axis=0)

In [ ]:
normMbDF.sum(axis=1).head()

In [ ]:
file2='%s/MicrobiomeDataTables/ForAnalysis/normMbDF' %MyPath
normMbDF.to_pickle(file2)

## only more than 0.0001:

In [ ]:
file1='%s/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA_s_AllSeqProjects_SampleListPNP515_filterGenotekTrue_filterMinimalRead9000000\
_libMethNone_noOutlierMeannSTD5nMinSamples3_fOutlierSampleFalse_fSamePersonFalse' %MyPath
groupedFilteredMB=pd.read_pickle(file1)

In [ ]:
groupedFilteredMB0001=groupedFilteredMB.mask(groupedFilteredMB<0.0001)
groupedFilteredMB0001=groupedFilteredMB0001.fillna(0)
groupedFilteredMB0001.head()


In [ ]:
checkShared=groupedFilteredMB0001.apply(lambda x: True if len([i for i in x if i>0])>1 else False)
shared=checkShared[checkShared]
sharedgroupedFilteredMB0001=groupedFilteredMB0001[shared.index]
print sharedgroupedFilteredMB0001.shape
sharedgroupedFilteredMB0001.head()

In [ ]:
file3='%s/MicrobiomeDataTables/ForAnalysis/MbDF_00001shared' %MyPath
sharedgroupedFilteredMB0001.to_pickle(file3)

In [ ]:
sharedgroupedFilteredMB0001_normed=sharedgroupedFilteredMB0001.div(sharedgroupedFilteredMB0001.sum(axis=1),axis=0)

In [ ]:
sharedgroupedFilteredMB0001_normed.sum(axis=1).head()

In [ ]:
file2='%s/MicrobiomeDataTables/ForAnalysis/MbDF_00001shared_normed' %MyPath
sharedgroupedFilteredMB0001_normed.to_pickle(file2)

## cap frequencies at 0.0001

In [ ]:
# df[df > .01].min(axis=1)

groupedFilteredMB[groupedFilteredMB>0].min().mean()

In [ ]:
# df = df.mask(df < df.quantile())
cappedMbDF=groupedFilteredMB.mask(groupedFilteredMB<0.0001)
cappedMbDF=cappedMbDF.fillna(0.0001)
cappedMbDF.head()

## remove outliers (std>5) using trimming

In [ ]:
cappedMbDF.head()

In [ ]:
cappedMbDF.index

In [ ]:
def filter_outliers(df,outlierSTD,columnList=None,trim=None):
    
    df2=pd.DataFrame()
    if columnList is None:
        columnList=df.columns.values
    for column in columnList:
        columnMean=df[column].mean()
        columnSTD=df[column].std()
        upperThreshold=columnMean+outlierSTD*columnSTD
        bottomThreshold=columnMean-outlierSTD*columnSTD
        
        if trim:
            bottomValue=bottomThreshold
            upperValue=upperThreshold
        else:
            bottomValue=np.nan
            upperValue=np.nan
#         print column,columnMean,columnSTD,bottomThreshold,upperThreshold
        df2[column]=np.where(df[column]<bottomThreshold,bottomValue,df[column])
        df2[column]=np.where(df[column]>upperThreshold,upperValue,df[column])
        df2.index=df.index
    return df2

In [ ]:
cappedTrimmedMbdf=filter_outliers(df=cappedMbDF,outlierSTD=5,columnList=None,trim=True)

In [ ]:
cappedTrimmedMbdf.head()

In [ ]:
cappedTrimmedMbdf.shape

## leave only shared species (treat 0.0001 as 0)

In [ ]:
checkShared=cappedTrimmedMbdf.apply(lambda x: True if len([i for i in x if i>0.0001])>1 else False)
shared=checkShared[checkShared]
sharedCappedTrimmedMbdf=cappedTrimmedMbdf[shared.index]
print sharedCappedTrimmedMbdf.shape
sharedCappedTrimmedMbdf.head()


### save file:

In [ ]:
file1='%s/MicrobiomeDataTables/ForAnalysis/sharedCappedTrimmedMbdf' %MyPath
sharedCappedTrimmedMbdf.to_pickle(file1)

## optional: normalize df:

In [ ]:
normSharedCappedTrimmedMbdf=sharedCappedTrimmedMbdf.div(sharedCappedTrimmedMbdf.sum(axis=1),axis=0)

In [ ]:
sharedCappedTrimmedMbdf.sum(axis=1).head()

In [ ]:
normSharedCappedTrimmedMbdf.sum(axis=1).head()

### save file:

In [ ]:
file1='%s/MicrobiomeDataTables/ForAnalysis/normSharedCappedTrimmedMbdf' %MyPath
normSharedCappedTrimmedMbdf.to_pickle(file1)

# prepare Mb matrix for distance correlation analysis - repeat for genus

## prepare MbDF table (mergedOnBD using noOutlierMean

In [ ]:
#load PNP515 sample list:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP515','rb') as fp:
    PNP515=pickle.load(fp)
    
#DEFINE PARAMETERS:
mbLevel='g'
mbDataFolder='AllSeqProjects'
SampleList=PNP515
SampleListName='PNP515'
filterGenotek=True
filterMinimalReads=9000000
filterlibPrepMethod=None
groupFunction='noOutlierMean'
nSTD=5
nMinSamples=3

the function genFilterMergeMbData was copied to TCR_microbiome_interactions_functions.py

In [ ]:
groupedFilteredMB=genFilterMergeMbData(mbLevel,mbDataFolder,SampleList,SampleListName,filterGenotek,filterMinimalReads,filterlibPrepMethod,
                        groupFunction,nSTD,nMinSamples)

In [ ]:
groupedFilteredMB.head()

## normalize MbDFold:

In [ ]:
groupedFilteredMB.sum().sum()

In [ ]:
groupedFilteredMB.head()

In [ ]:
normMbDF=groupedFilteredMB.div(groupedFilteredMB.sum(axis=1),axis=0)

In [ ]:
normMbDF.sum(axis=1).head()

In [ ]:
file2='%s/MicrobiomeDataTables/ForAnalysis/normMbDFgenus' %MyPath
normMbDF.to_pickle(file2)

## only more than 0.0001:

In [ ]:
groupedFilteredMB0001=groupedFilteredMB.mask(groupedFilteredMB<0.0001)
groupedFilteredMB0001=groupedFilteredMB0001.fillna(0)
groupedFilteredMB0001.head()


In [ ]:
checkShared=groupedFilteredMB0001.apply(lambda x: True if len([i for i in x if i>0])>1 else False)
shared=checkShared[checkShared]
sharedgroupedFilteredMB0001=groupedFilteredMB0001[shared.index]
print sharedgroupedFilteredMB0001.shape
sharedgroupedFilteredMB0001.head()

In [ ]:
file3='%s/MicrobiomeDataTables/ForAnalysis/MbDFgenus_00001shared' %MyPath
sharedgroupedFilteredMB0001.to_pickle(file3)

In [ ]:
sharedgroupedFilteredMB0001_normed=sharedgroupedFilteredMB0001.div(sharedgroupedFilteredMB0001.sum(axis=1),axis=0)

In [ ]:
sharedgroupedFilteredMB0001_normed.sum(axis=1).head()

In [ ]:
file2='%s/MicrobiomeDataTables/ForAnalysis/MbDFgenus_00001shared_normed' %MyPath
sharedgroupedFilteredMB0001_normed.to_pickle(file2)

## cap frequencies at 0.0001

In [ ]:
# df[df > .01].min(axis=1)

groupedFilteredMB[groupedFilteredMB>0].min().mean()

In [ ]:
# df = df.mask(df < df.quantile())
cappedMbDF=groupedFilteredMB.mask(groupedFilteredMB<0.0001)
cappedMbDF=cappedMbDF.fillna(0.0001)
cappedMbDF.head()

## remove outliers (std>5) using trimming

In [ ]:
cappedTrimmedMbdf=filter_outliers(df=cappedMbDF,outlierSTD=5,columnList=None,trim=True)

In [ ]:
cappedTrimmedMbdf.shape

## leave only shared species (treat 0.0001 as 0)

In [ ]:
checkShared=cappedTrimmedMbdf.apply(lambda x: True if len([i for i in x if i>0.0001])>1 else False)
shared=checkShared[checkShared]
sharedCappedTrimmedMbdf=cappedTrimmedMbdf[shared.index]
print sharedCappedTrimmedMbdf.shape
sharedCappedTrimmedMbdf.head()


### save file:

In [ ]:
file1='%s/MicrobiomeDataTables/ForAnalysis/sharedCappedTrimmedMbdf_genus' %MyPath
sharedCappedTrimmedMbdf.to_pickle(file1)

## optional: normalize df:

In [ ]:
normSharedCappedTrimmedMbdf=sharedCappedTrimmedMbdf.div(sharedCappedTrimmedMbdf.sum(axis=1),axis=0)

In [ ]:
sharedCappedTrimmedMbdf.sum(axis=1).head()

In [ ]:
normSharedCappedTrimmedMbdf.sum(axis=1).head()

### save file:

In [ ]:
file1='%s/MicrobiomeDataTables/ForAnalysis/normSharedCappedTrimmedMbdf_genus' %MyPath
normSharedCappedTrimmedMbdf.to_pickle(file1)

# compare MB and TCR histograms for some samples

In [ ]:
file1='%s/MicrobiomeDataTables/ForAnalysis/normSharedCappedTrimmedMbdf' %MyPath
MbDF=pd.read_pickle(file1)
MbDF.head()

In [ ]:
file3='%s/MicrobiomeDataTables/ForAnalysis/normSharedCappedTrimmedMbdf_genus' %MyPath
MbDFgenus=pd.read_pickle(file3)
MbDFgenus.head()

In [ ]:
file2='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA_onlyProductive%s_divided_trimmed_capped_normed' %(dataset_folder,datasetName,minNshared,
                                                                               onlyProductive)
TCRdf=pd.read_pickle(file2)
TCRdf.head()

In [ ]:
TCRdfmax=TCRdf.max(axis=1).sort_values(ascending=False)
TCRdfmax

In [ ]:
MbDF.max(axis=1).sort_values(ascending=False).head()

In [ ]:
MbDFgenus.max(axis=1).sort_values(ascending=False).head()

In [ ]:
TCRdf.loc['BD19',:].sum()

In [ ]:
fig, axes=plt.subplots(nrows=3,ncols=4, figsize=(32,15))

# print axes

sampleList=['BD696','BD3','BD461','BD428']

for n,sample in enumerate(sampleList):
    ax1=axes[0,n]
    ax1.hist(list(TCRdf.loc[sample,:]),bins=100,color='red',alpha=0.5)
    ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
    ax1.set_yscale('log')
    ax1.set_title(sample)
    ax1.set_xlabel('TCR RA')
    ax1.set_ylabel('Frequency')
    ax1.set_xlim(0,0.002)
    
    ax2=axes[1,n]
    ax2.hist(list(MbDF.loc[sample,:]),bins=100,color='blue',alpha=0.5)
    ax2.xaxis.set_major_locator(plt.MaxNLocator(5))
    ax2.set_yscale('log')
    ax2.set_title(sample)
    ax2.set_xlabel('Species RA')
    ax2.set_ylabel('Frequency')
    ax2.set_xlim(0,0.08)

    ax3=axes[2,n]
    ax3.hist(list(MbDFgenus.loc[sample,:]),bins=100,color='blue',alpha=0.5)
    ax3.xaxis.set_major_locator(plt.MaxNLocator(5))
    ax3.set_yscale('log')
    ax3.set_title(sample)
    ax3.set_xlabel('Genus RA')
    ax3.set_ylabel('Frequency')
    ax3.set_xlim(0,0.08)
    
    

In [ ]:
plt.show()

In [ ]:
file4='%s/TCR_mb_results/distCorrelations/TCR&MbFreq' %dataset_folder
fig.savefig(file4, dpi=200)

# generate distance matrices

## define functions:

In [ ]:
the functions genDistMat and genDistMatsForAll were copied to feature_phenotype_functions.py

In [ ]:
def genDistMatsForAll(df,dfName):
    TCR_distMatFolder='%s/distanceMatrices' %dataset_folder
    MB_distMatFolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath
    
    print 'generating distance matrix for RA data, df=%s' %dfName
    metric1='braycurtis'
    df_condensed_org, df_distMat_square=genDistMat(df,metric=metric1)
    
    print 'saving RA distMat CONDENSED'
    
    df_condensed_org.to_pickle(file1)
    
    if 'Mbdf' in dfName:
        folder=MB_distMatFolder   
    else:
        folder=TCR_distMatFolder
    
    print 'saving RA distMat'
    file1='%s/%s_%s_distMat' %(folder,dfName,metric1)
    file2='%s/%s_%s_distMat_CONDENSED' %(folder,dfName,metric1)
    df_distMat_square.to_pickle(file1)
    df_condensed_org.to_pickle(file2)
    
    if 'norm' not in dfName:
        print 'converting data to binary'
        minVal=df.min().min()
        print 'the min value of this df is %s' %minVal
        df2=df.copy()
        df2=(df2>minVal).astype(int)
        print df2.iloc[:3,:3]

        print 'saving binary df...'
        file2='%s/%s_binary' %(folder,dfName)
        df2.to_pickle(file2)

        print 'generating binary distance metric, df=%s' %dfName
        metric2='jaccard'

        df_condensed_org_binary, df_distMat_square_binary=genDistMat(df2,metric=metric2)
        print 'saving binary distMat'
        file3='%s/%s_binary_%s_distMat' %(folder,dfName,metric2)
        df_distMat_square_binary.to_pickle(file3)
    else:
        df_distMat_square_binary=pd.DataFrame()
        print 'No binary df for normed df!'
        
    print 'DONE!'
    
    return df_distMat_square,df_distMat_square_binary
    

## run trial:

In [ ]:
file1='%s/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA_s_AllSeqProjects_SampleListPNP515_filterGenotekTrue_filterMinimalRead9000000_libMethNone_noOutlierMeannSTD5nMinSamples3_fOutlierSampleFalse_fSamePersonFalse' %MyPath

df=pd.read_pickle(file1)
dfName='MbdfOld'

df_distMat_square,df_distMat_square_binary=genDistMatsForAll(df,dfName)

## run for all TCR and MB dfs:

In [ ]:
dfNameList=['normSharedCappedTrimmedMbdf','normSharedCappedTrimmedMbdf_genus','sharedCappedTrimmedMbdf',
           'sharedCappedTrimmedMbdf_genus','TCRonlyProductiveTrue',
           'TCRonlyProductiveTrue_divided_trimmed','TCRonlyProductiveTrue_divided_trimmed_capped',
           'TCRonlyProductiveTrue_divided_trimmed_capped_normed']

for n,dfName in enumerate(dfNameList):
    print n,dfName
    
    if 'Mbdf' in dfName:
        folder='%s/MicrobiomeDataTables/ForAnalysis' %MyPath
        file1='%s/%s' %(folder, dfName)
    else:
        folder='%s/sharingAnalysis' %dataset_folder
        f=dfName.replace('TCR','')
        file1='%s/sharingMatrix_PNP515_ss15000_rep2_minNshared2_RA_%s' %(folder, f)

    df=pd.read_pickle(file1)

    df_distMat_square,df_distMat_square_binary=genDistMatsForAll(df,dfName)


## run for new TCR and MB dfs:

In [ ]:
dfNameList=['normMbDF','MbDF_00001shared','MbDF_00001shared_normed',
            'MbdfOldgenus','normMbDFgenus','MbDFgenus_00001shared','MbDFgenus_00001shared_normed',
            'TCRonlyProductiveTrue_normed','TCRonlyProductiveTrue_2copiesshared','TCRonlyProductiveTrue_2copiesshared_normed']

for n,dfName in enumerate(dfNameList):
    print n,dfName
    
    if ('Mbdf' in dfName) or ('MbDF' in dfName):
        folder='%s/MicrobiomeDataTables/ForAnalysis' %MyPath
        file1='%s/%s' %(folder, dfName)
    else:
        folder='%s/sharingAnalysis' %dataset_folder
        f=dfName.replace('TCR','')
        file1='%s/sharingMatrix_PNP515_ss15000_rep2_minNshared2_RA_%s' %(folder, f)

    df=pd.read_pickle(file1)

    df_distMat_square,df_distMat_square_binary=genDistMatsForAll(df,dfName)

# run mantel

## run for all binary distMat pairs:

In [ ]:
method='spearman'
n_permut=999
alternative='greater'
removeSameUser=True

In [ ]:
MantelList=[('TCRonlyProductiveTrue_binary_jaccard_distMat','MbdfOld_binary_jaccard_distMat'),
           ('TCRonlyProductiveTrue_binary_jaccard_distMat','sharedCappedTrimmedMbdf_binary_jaccard_distMat'),
           ('TCRonlyProductiveTrue_binary_jaccard_distMat','sharedCappedTrimmedMbdf_genus_binary_jaccard_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_binary_jaccard_distMat','sharedCappedTrimmedMbdf_binary_jaccard_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_binary_jaccard_distMat','sharedCappedTrimmedMbdf_genus_binary_jaccard_distMat'),
           ('TCRonlyProductiveTrue_divided_trimmed_capped_binary_jaccard_distMat','sharedCappedTrimmedMbdf_binary_jaccard_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_capped_binary_jaccard_distMat','sharedCappedTrimmedMbdf_genus_binary_jaccard_distMat')]
            

for n,mantel in enumerate(MantelList):
    
    print n,mantel
    
    distMat1name=mantel[0]
    distMat2name=mantel[1]
    
    TCR_distMatFolder='%s/distanceMatrices' %dataset_folder
    MB_distMatFolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath
    
    TCRfile='%s/%s' %(TCR_distMatFolder,distMat1name)
    MBfile='%s/%s' %(MB_distMatFolder,distMat2name)
    
    distMat1=pd.read_pickle(TCRfile)
    distMat2=pd.read_pickle(MBfile)
    
    MantelFolder='%s/TCR_mb_results/distCorrelations/MantelDFs' %dataset_folder
    if not isdir(MantelFolder):
        makedirs(MantelFolder)
    
    mantelDF=mantelTest_feature_phenotype(distMat1, distMat2,distMat1name, distMat2name, MantelFolder,method,n_permut,alternative,removeSameUser)
    mantelDF.loc[0,'datasetName']=datasetName
    mantelDF.loc[0,'removeSameUser']=removeSameUser
    
    mantelFile='%s/%s_%s' %(MantelFolder,distMat1name,distMat2name)
    mantelDF.to_pickle(mantelFile)
    

## run for all RA distMat pairs:

In [ ]:
method='spearman'
n_permut=999
alternative='greater'
removeSameUser=True

In [ ]:
MantelList=[('TCRonlyProductiveTrue_braycurtis_distMat','MbdfOld_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_braycurtis_distMat','sharedCappedTrimmedMbdf_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_braycurtis_distMat','sharedCappedTrimmedMbdf_genus_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_braycurtis_distMat','sharedCappedTrimmedMbdf_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_braycurtis_distMat','sharedCappedTrimmedMbdf_genus_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_capped_braycurtis_distMat','sharedCappedTrimmedMbdf_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_capped_braycurtis_distMat','sharedCappedTrimmedMbdf_genus_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_capped_normed_braycurtis_distMat','normSharedCappedTrimmedMbdf_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_capped_normed_braycurtis_distMat','normSharedCappedTrimmedMbdf_genus_braycurtis_distMat')]
            

for n,mantel in enumerate(MantelList):
    
    print n,mantel
    
    distMat1name=mantel[0]
    distMat2name=mantel[1]
    
    TCR_distMatFolder='%s/distanceMatrices' %dataset_folder
    MB_distMatFolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath
    
    TCRfile='%s/%s' %(TCR_distMatFolder,distMat1name)
    MBfile='%s/%s' %(MB_distMatFolder,distMat2name)
    
    distMat1=pd.read_pickle(TCRfile)
    distMat2=pd.read_pickle(MBfile)
    
    MantelFolder='%s/TCR_mb_results/distCorrelations/MantelDFs' %dataset_folder
    if not isdir(MantelFolder):
        makedirs(MantelFolder)
    
    mantelDF=mantelTest_feature_phenotype(distMat1, distMat2,distMat1name, distMat2name, MantelFolder,method,n_permut,alternative,removeSameUser)
    mantelDF.loc[0,'datasetName']=datasetName
    mantelDF.loc[0,'removeSameUser']=removeSameUser
    
    mantelFile='%s/%s_%s' %(MantelFolder,distMat1name,distMat2name)
    mantelDF.to_pickle(mantelFile)
    

## New mantel tests - RA

In [ ]:
method='spearman'
n_permut=999
alternative='greater'
removeSameUser=True

In [ ]:
MantelList=[('TCRonlyProductiveTrue_normed_braycurtis_distMat','normMbDF_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_normed_braycurtis_distMat','normMbDFgenus_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_braycurtis_distMat','MbdfOld_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_braycurtis_distMat','MbdfOldgenus_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_2copiesshared_braycurtis_distMat','MbDF_00001shared_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_2copiesshared_braycurtis_distMat','MbDFgenus_00001shared_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_2copiesshared_normed_braycurtis_distMat','MbDF_00001shared_normed_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_2copiesshared_normed_braycurtis_distMat','MbDFgenus_00001shared_normed_braycurtis_distMat')]
            
            
            
for n,mantel in enumerate(MantelList):
    
    print n,mantel
    
    distMat1name=mantel[0]
    distMat2name=mantel[1]
    
    TCR_distMatFolder='%s/distanceMatrices' %dataset_folder
    MB_distMatFolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath
    
    TCRfile='%s/%s' %(TCR_distMatFolder,distMat1name)
    MBfile='%s/%s' %(MB_distMatFolder,distMat2name)
    
    distMat1=pd.read_pickle(TCRfile)
    distMat2=pd.read_pickle(MBfile)
    
    MantelFolder='%s/TCR_mb_results/distCorrelations/MantelDFs' %dataset_folder
    if not isdir(MantelFolder):
        makedirs(MantelFolder)
    
    mantelDF=mantelTest_feature_phenotype(distMat1, distMat2,distMat1name, distMat2name, MantelFolder,method,n_permut,alternative,removeSameUser)
    mantelDF.loc[0,'datasetName']=datasetName
    mantelDF.loc[0,'removeSameUser']=removeSameUser
    
    mantelFile='%s/%s_%s' %(MantelFolder,distMat1name,distMat2name)
    mantelDF.to_pickle(mantelFile)

## New mantel tests - binary

In [ ]:
method='spearman'
n_permut=999
alternative='greater'
removeSameUser=True

In [ ]:
MantelList=[('TCRonlyProductiveTrue_binary_jaccard_distMat','MbdfOldgenus_binary_jaccard_distMat'),
            ('TCRonlyProductiveTrue_binary_jaccard_distMat','MbdfOld_binary_jaccard_distMat'),
            ('TCRonlyProductiveTrue_2copiesshared_binary_jaccard_distMat','MbDF_00001shared_binary_jaccard_distMat'),
            ('TCRonlyProductiveTrue_2copiesshared_binary_jaccard_distMat','MbDFgenus_00001shared_binary_jaccard_distMat')]
            
            
            
for n,mantel in enumerate(MantelList):
    
    print n,mantel
    
    distMat1name=mantel[0]
    distMat2name=mantel[1]
    
    TCR_distMatFolder='%s/distanceMatrices' %dataset_folder
    MB_distMatFolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath
    
    TCRfile='%s/%s' %(TCR_distMatFolder,distMat1name)
    MBfile='%s/%s' %(MB_distMatFolder,distMat2name)
    
    distMat1=pd.read_pickle(TCRfile)
    distMat2=pd.read_pickle(MBfile)
    
    MantelFolder='%s/TCR_mb_results/distCorrelations/MantelDFs' %dataset_folder
    if not isdir(MantelFolder):
        makedirs(MantelFolder)
    
    mantelDF=mantelTest_feature_phenotype(distMat1, distMat2,distMat1name, distMat2name, MantelFolder,method,n_permut,alternative,removeSameUser)
    mantelDF.loc[0,'datasetName']=datasetName
    mantelDF.loc[0,'removeSameUser']=removeSameUser
    
    mantelFile='%s/%s_%s' %(MantelFolder,distMat1name,distMat2name)
    mantelDF.to_pickle(mantelFile)

## run again excluding problematic samples

In [7]:
file1='%s/Sample files/BDnoSameYearMatch.xlsx' %MyPath
BDnoSameYearMatch=gen_py_BD_list(file1)
print BDnoSameYearMatch[:5]
print len(BDnoSameYearMatch)

['BD133', 'BD139', 'BD144', 'BD153', 'BD188']
36


In [8]:
method='spearman'
n_permut=999
alternative='greater'
removeSameUser=True

In [15]:
MantelList=[('TCRonlyProductiveTrue_braycurtis_distMat','MbdfOld_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_braycurtis_distMat','sharedCappedTrimmedMbdf_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_braycurtis_distMat','sharedCappedTrimmedMbdf_genus_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_braycurtis_distMat','sharedCappedTrimmedMbdf_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_braycurtis_distMat','sharedCappedTrimmedMbdf_genus_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_capped_braycurtis_distMat','sharedCappedTrimmedMbdf_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_capped_braycurtis_distMat','sharedCappedTrimmedMbdf_genus_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_capped_normed_braycurtis_distMat','normSharedCappedTrimmedMbdf_braycurtis_distMat'),
            ('TCRonlyProductiveTrue_divided_trimmed_capped_normed_braycurtis_distMat','normSharedCappedTrimmedMbdf_genus_braycurtis_distMat')]
            

for n,mantel in enumerate(MantelList):
    
    print n,mantel
    
    distMat1name=mantel[0]
    distMat2name=mantel[1]
    
    
    
    TCR_distMatFolder='%s/distanceMatrices' %dataset_folder
    MB_distMatFolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath
    
    TCRfile='%s/%s' %(TCR_distMatFolder,distMat1name)
    MBfile='%s/%s' %(MB_distMatFolder,distMat2name)
    
    
    distMat1=pd.read_pickle(TCRfile)
    distMat2=pd.read_pickle(MBfile)
    
    print 'removing problematic samples...'
    print 'distMat1 shape before samples removal is %s_%s' %(distMat1.shape[0],distMat1.shape[1])
    print 'distMat2 shape before samples removal is %s_%s' %(distMat2.shape[0],distMat2.shape[1])
    
    for sample in BDnoSameYearMatch:
#         print sample
        if sample in list(distMat1.index):
#             print 'samples %s is in distMat1 index' %sample
            distMat1=distMat1.drop(sample,axis=0)
            distMat1=distMat1.drop(sample,axis=1)
        if sample in list(distMat2.index):
#             print 'samples %s is in distMat2 index' %sample
            distMat2=distMat2.drop(sample,axis=0)
            distMat2=distMat2.drop(sample,axis=1)

    
    print 'distMat1 shape AFTER samples removal is %s_%s' %(distMat1.shape[0],distMat1.shape[1])
    print 'distMat2 shape AFTER samples removal is %s_%s' %(distMat2.shape[0],distMat2.shape[1])
    
    MantelFolder='%s/TCR_mb_results/distCorrelations/MantelDFs' %dataset_folder
    if not isdir(MantelFolder):
        makedirs(MantelFolder)
    
    mantelDF=mantelTest_feature_phenotype(distMat1, distMat2,distMat1name, distMat2name, MantelFolder,method,n_permut,alternative,removeSameUser)
    mantelDF.loc[0,'datasetName']=datasetName
    mantelDF.loc[0,'removeSameUser']=removeSameUser
    mantelDF.loc[0,'noProblematic']='True'
    
    
    mantelFile='%s/%s_%s_noProblematic' %(MantelFolder,distMat1name,distMat2name)
    mantelDF.to_pickle(mantelFile)
    

0 ('TCRonlyProductiveTrue_braycurtis_distMat', 'MbdfOld_braycurtis_distMat')
removing problematic samples...
distMat1 shape before samples removal is 392_392
distMat2 shape before samples removal is 447_447
distMat1 shape AFTER samples removal is 373_373
distMat2 shape AFTER samples removal is 428_428
original x array shape is 373_373
original y array shape is 428_428
loading and processing distance matrix files...
verifying same samples in compared distance matrices:
filtering same person samples
loading BD_FD table
number of BD-UserID pairs is 1099
merging UserID info to BDgrouped table:
0 UserIDs have more than one BD
groupedDF length before dropping same person samples is 321
removing UserIDs: []
removing BD: []
groupedDF shape after dropping same person samples is 321_321
loading BD_FD table
number of BD-UserID pairs is 1099
merging UserID info to BDgrouped table:
0 UserIDs have more than one BD
groupedDF length before dropping same person samples is 321
removing UserIDs: []
remov

0.016829416425405107 0.266
saving mantel results
DONE!
4 ('TCRonlyProductiveTrue_divided_trimmed_braycurtis_distMat', 'sharedCappedTrimmedMbdf_genus_braycurtis_distMat')
removing problematic samples...
distMat1 shape before samples removal is 392_392
distMat2 shape before samples removal is 447_447
distMat1 shape AFTER samples removal is 373_373
distMat2 shape AFTER samples removal is 428_428
original x array shape is 373_373
original y array shape is 428_428
loading and processing distance matrix files...
verifying same samples in compared distance matrices:
filtering same person samples
loading BD_FD table
number of BD-UserID pairs is 1099
merging UserID info to BDgrouped table:
0 UserIDs have more than one BD
groupedDF length before dropping same person samples is 321
removing UserIDs: []
removing BD: []
groupedDF shape after dropping same person samples is 321_321
loading BD_FD table
number of BD-UserID pairs is 1099
merging UserID info to BDgrouped table:
0 UserIDs have more than 

0.04121868539753553 0.128
saving mantel results
DONE!
8 ('TCRonlyProductiveTrue_divided_trimmed_capped_normed_braycurtis_distMat', 'normSharedCappedTrimmedMbdf_genus_braycurtis_distMat')
removing problematic samples...
distMat1 shape before samples removal is 392_392
distMat2 shape before samples removal is 447_447
distMat1 shape AFTER samples removal is 373_373
distMat2 shape AFTER samples removal is 428_428
original x array shape is 373_373
original y array shape is 428_428
loading and processing distance matrix files...
verifying same samples in compared distance matrices:
filtering same person samples
loading BD_FD table
number of BD-UserID pairs is 1099
merging UserID info to BDgrouped table:
0 UserIDs have more than one BD
groupedDF length before dropping same person samples is 321
removing UserIDs: []
removing BD: []
groupedDF shape after dropping same person samples is 321_321
loading BD_FD table
number of BD-UserID pairs is 1099
merging UserID info to BDgrouped table:
0 UserID

## merge Mantel results

In [16]:
MantelFolder='%s/TCR_mb_results/distCorrelations/MantelDFs' %dataset_folder
file1='%s/TCRonlyProductiveTrue_2copiesshared_binary_jaccard_distMat_MbDF_00001shared_binary_jaccard_distMat' %MantelFolder
mantel1=pd.read_pickle(file1)
mantel1

,distMat1name,distMat2name,method,n_permut,alternative,corr_coeff,p_value,N,datasetName,removeSameUser
0,TCRonlyProductiveTrue_2copiesshared_binary_jac...,MbDF_00001shared_binary_jaccard_distMat,spearman,999.0,greater,-0.002018,0.541,336.0,PNP515_ss15000_rep2,True


In [17]:
MantelFolder='%s/TCR_mb_results/distCorrelations/MantelDFs' %dataset_folder
MantelResults=concat_summarizing_dfs_excel(MantelFolder)

number of dfs in directory: 35
the length of df list is 35


In [18]:
MantelResults=MantelResults.sort_values(by='p_value')
MantelResults=add_corrected_pValues(MantelResults,pValueColumn='p_value',nTests=len(MantelResults),FDR=0.1)

In [19]:
file1='%s/TCR_mb_results/distCorrelations/MantelResults290518.xlsx' %dataset_folder
MantelResults.to_excel(file1)

# plot correlations, with relationships colors

# continue to edit TCRdf and MbDF for specific seq-species pair analysis

# run seq-species pair analysis (no permutations)